In [1]:
import json
import pandas as pd

In [2]:
all_cards = json.load(open("../data/AllPrintings.json"))

In [ ]:
url = "https://api.scryfall.com/" + "cards/collection"
sesh = Session()
sesh.headers.update({'Content-Type' : 'application/json'})

In [ ]:
parameters = {
    'identifiers' : [{'name' : 'Nicol Bolas, God-Pharaoh (Foil Etched)',
                      'set' : '2X2'}]
}

In [32]:
my_cards = pd.read_csv("../data/my_collection.csv")

In [33]:
my_cards = my_cards[['name', 'set_code', 'treatment', 'foil', 'ID']]

In [34]:
my_cards

,name,set_code,treatment,foil,ID
0,"Nicol Bolas, God-Pharaoh",2X2,Foil Etched,1,45109693
1,Dockside Extortionist,2X2,Borderless,1,45049212
2,"Ulamog, the Infinite Gyre",2X2,Borderless,1,45049210
3,"Ulamog, the Infinite Gyre",2X2,Borderless,0,45049209
4,Grief,mh2,NaN,0,44954460
...,...,...,...,...,...
630,Arid Mesa,mh2,NaN,0,39559026
631,Dauthi Voidwalker,MH2,Retro Frame,0,39559043
632,Sylvan Anthem,mh2,NaN,1,39559056
633,Sol Talisman,mh2,NaN,1,39559084


In [45]:
just_double_masters = my_cards[my_cards.set_code == "2X2"]

In [46]:
just_double_masters

,name,set_code,treatment,foil,ID
0,"Nicol Bolas, God-Pharaoh",2X2,Foil Etched,1,45109693
1,Dockside Extortionist,2X2,Borderless,1,45049212
2,"Ulamog, the Infinite Gyre",2X2,Borderless,1,45049210
3,"Ulamog, the Infinite Gyre",2X2,Borderless,0,45049209


In [35]:
my_cards.set_code = my_cards.set_code.str.upper()

In [37]:
my_cards.set_code.unique()

array(['2X2', 'MH2', 'DPA', 'USG', 'MID', 'WAR', 'RNA', 'CLB', 'MH1',
       'DKA', 'GRN', 'KLD', 'TSR', 'ZNR', 'IKO', 'AFR', 'KHM', '2XM',
       'M19', 'ORI', 'RIX', 'M20', 'MM2', 'SOI', 'OGW', 'M21', 'VOW',
       'SNC', 'STX', 'ODY', 'MM3', 'RTR', 'NEO', 'PLIST', 'PPRE', 'C15',
       'DDS', 'NCC', 'A25', 'ARB', 'ELD', 'IMA', 'NPH', 'AFC', 'MMQ',
       'EMN', 'MMA', 'SLD', 'ISD', 'ICE', 'AER', 'BAB', 'NEC', 'BNG',
       'PAFR', 'EMA', 'PD3', 'MIC', 'ZNC', 'THS', 'C17', 'PIKO', 'ZEN',
       'V10', 'STA', 'FUT', 'ZNE', 'ONS', 'AVR', 'SHM', 'V11', 'STH',
       'PSTX'], dtype=object)

In [38]:
double_masters = all_cards['data']['2X2']['cards']

In [42]:
masters_df = pd.DataFrame(double_masters)

In [43]:
masters_df.name

0        Emrakul, the Aeons Torn
1      Kozilek, Butcher of Truth
2      Ulamog, the Infinite Gyre
3                 Abzan Falconer
4                 Ainok Bond-Kin
                 ...            
574      Emrakul, the Aeons Torn
575    Kozilek, Butcher of Truth
576    Ulamog, the Infinite Gyre
577           Weathered Wayfarer
578               Bring to Light
Name: name, Length: 579, dtype: object

In [50]:
one_card = just_double_masters.iloc[0]

In [117]:
# now need to write out the logic for searching through the sets
# double masters will have 2 different effects, etched foil and borderless
def where_to_look(treatment):
    temp = treatment.lower()
    
    # borderless -> borderColor
    # etched foil -> finishes
    # extended art -> frameEffects
    # retro frame -> frameVersion
    # retro + foil etched -> frameVersion + higher collector number
    # JP Alternate Art -> ?
    # Showcase -> ?
    # Gilded Foil -> ?
    # Dungeon Module -> ?
    
    conversion_dict = {
        'borderless' : ['borderColor', ['borderless']],
        'etched foil' : ['finishes', ['etched']],
        'extended art' : ['frameEffects', ['???']],
        'retro frame' : ['frameVersion', ['1997']],
        'jp alternate art' : ['???', ['???']],
        'showcase' : ['???', ['???']],
        'gilded foil' : ['???', ['???']],
        'dungeon module' : ['???', ['???']]}
        
    


def fetch_setnumber(card_info, referenceDB):
    '''
    we want to do this for individual cards and then we can expand to more
    '''
    
    # first part is to determine what treatment the card is so we can then
    # influence where we are looking in the main reference db
    
    treatment = card_info.treatment.lower()
    
    conversion_dict = {
        'borderless' : ['borderColor', 'borderless'],
        'foil etched' : ['finishes', ['etched']],
        'extended art' : ['frameEffects', ['???']],
        'retro frame' : ['frameVersion', ['1997']],
        'jp alternate art' : ['???', ['???']],
        'showcase' : ['???', ['???']],
        'gilded foil' : ['???', ['???']],
        'dungeon module' : ['???', ['???']]}
    
    relevant_keys = conversion_dict[treatment]
    
    print(relevant_keys)
    
    filtered = referenceDB[referenceDB.name == card_info['name']]
    
    print(filtered)
    
    result = filtered[filtered[relevant_keys[0]].isin([relevant_keys[1]])]
    
    print(result.number)

    return int(result.number.iloc[0])

In [116]:
keys = ['borderColor', 'borderless']
card_inf = just_double_masters.iloc[1]
print(card_inf)
filt = masters_df[masters_df.name == card_inf['name']]
print(keys[1])
filt[filt[keys[0]].isin([keys[1]])]

name         Dockside Extortionist
set_code                       2X2
treatment               Borderless
foil                             1
ID                        45049212
Name: 1, dtype: object
borderless


,artist,availability,boosterTypes,borderColor,colorIdentity,colors,convertedManaCost,finishes,foreignData,frameEffects,...,types,uuid,variations,edhrecRank,flavorText,loyalty,hasAlternativeDeckLimit,isStarter,promoTypes,isPromo
359,Douglas Shuler,[paper],NaN,borderless,[R],[R],2.0,"[nonfoil, foil]",[],[inverted],...,[Creature],110e193d-f86b-59f0-a979-fd0615c7576e,"[1104bdaa-8841-5b80-8300-0e8d0fcd33b7, 3db882c...",79.0,"The fine print is illegible scribbles, but it ...",NaN,NaN,True,[boosterfun],NaN


In [119]:
just_double_masters['set_number'] = just_double_masters.apply(lambda x: fetch_setnumber(x, masters_df), axis = 1)

['finishes', ['etched']]
               artist   availability boosterTypes borderColor colorIdentity  \
259  Raymond Swanland  [mtgo, paper]      [draft]       black     [B, R, U]   
529  Raymond Swanland        [paper]          NaN       black     [B, R, U]   

        colors  convertedManaCost         finishes foreignData frameEffects  \
259  [B, R, U]                7.0  [nonfoil, foil]          []          NaN   
529  [B, R, U]                7.0         [etched]          []   [inverted]   

     ...           types                                  uuid  \
259  ...  [Planeswalker]  6cac882b-57d3-5b6b-a3c9-56dcd5401f4b   
529  ...  [Planeswalker]  62278030-0f17-5cad-bdb7-9607a15bc898   

                                 variations edhrecRank flavorText loyalty  \
259  [62278030-0f17-5cad-bdb7-9607a15bc898]     2873.0        NaN       7   
529  [6cac882b-57d3-5b6b-a3c9-56dcd5401f4b]     2873.0        NaN       7   

    hasAlternativeDeckLimit isStarter    promoTypes isPromo  
259   

/var/folders/cn/9ny0n35x1p18h680wlz6lmzm0000gn/T/ipykernel_17521/2480686054.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  just_double_masters['set_number'] = just_double_masters.apply(lambda x: fetch_setnumber(x, masters_df), axis = 1)


In [121]:
just_double_masters

,name,set_code,treatment,foil,ID,set_number
0,"Nicol Bolas, God-Pharaoh",2X2,Foil Etched,1,45109693,530
1,Dockside Extortionist,2X2,Borderless,1,45049212,360
2,"Ulamog, the Infinite Gyre",2X2,Borderless,1,45049210,337
3,"Ulamog, the Infinite Gyre",2X2,Borderless,0,45049209,337


In [53]:
one_card['name']

'Nicol Bolas, God-Pharaoh'

In [69]:
filtered = masters_df[masters_df.name == one_card['name']]
filtered[filtered.finishes.isin([['etched']])]

,artist,availability,boosterTypes,borderColor,colorIdentity,colors,convertedManaCost,finishes,foreignData,frameEffects,...,types,uuid,variations,edhrecRank,flavorText,loyalty,hasAlternativeDeckLimit,isStarter,promoTypes,isPromo
529,Raymond Swanland,[paper],NaN,black,"[B, R, U]","[B, R, U]",7.0,[etched],[],[inverted],...,[Planeswalker],62278030-0f17-5cad-bdb7-9607a15bc898,[6cac882b-57d3-5b6b-a3c9-56dcd5401f4b],2873.0,NaN,7,NaN,NaN,[boosterfun],NaN


In [61]:
filtered

,artist,availability,boosterTypes,borderColor,colorIdentity,colors,convertedManaCost,finishes,foreignData,frameEffects,...,types,uuid,variations,edhrecRank,flavorText,loyalty,hasAlternativeDeckLimit,isStarter,promoTypes,isPromo


In [58]:
filtered.finishes[259]

['nonfoil', 'foil']

In [23]:
all_cards['data']['MH2']['cards'][438]

{'artist': 'Volkan Baǵa',
 'availability': ['mtgo', 'paper'],
 'borderColor': 'black',
 'colorIdentity': [],
 'colors': [],
 'convertedManaCost': 0.0,
 'edhrecRank': 1659,
 'finishes': ['nonfoil', 'foil', 'etched'],
 'flavorText': 'It contains the energy of a dying star, caught at the moment of its explosion.',
 'foreignData': [{'flavorText': 'Er enthält die Energie eines sterbenden Sterns, die im Moment seiner Explosion eingefangen wurde.',
   'language': 'German',
   'multiverseId': 525674,
   'name': 'Sonnentalisman',
   'text': 'Aussetzen 3 — {1}\n{T}: Erzeuge {C}{C}.',
   'type': 'Artefakt'},
  {'flavorText': 'Contiene la energía de una estrella moribunda atrapada en plena explosión.',
   'language': 'Spanish',
   'multiverseId': 525735,
   'name': 'Talismán solar',
   'text': 'Suspender 3—{1}.\n{T}: Agrega {C}{C}.',
   'type': 'Artefacto'},
  {'flavorText': "Il contient l'énergie d'une étoile mourante, capturée au moment de son explosion.",
   'language': 'French',
   'multiverse

In [13]:
all_cards['data']['2X2']['cards'][-200:-140][0]

{'artist': 'Richard Kane Ferguson',
 'availability': ['paper'],
 'borderColor': 'borderless',
 'colorIdentity': ['B', 'G'],
 'colors': ['B', 'G'],
 'convertedManaCost': 2.0,
 'edhrecRank': 8639,
 'finishes': ['nonfoil', 'foil'],
 'foreignData': [],
 'frameEffects': ['inverted'],
 'frameVersion': '2015',
 'hasFoil': True,
 'hasNonFoil': True,
 'identifiers': {'cardKingdomFoilId': '263416',
  'cardKingdomId': '263242',
  'mcmId': '664920',
  'mcmMetaId': '223073',
  'mtgjsonV4Id': '3f66ef43-6735-5efd-ae4e-9d74bb49e3b2',
  'multiverseId': '573042',
  'scryfallId': '356110b1-9cfe-420c-963a-c12469786e40',
  'scryfallIllustrationId': 'e96514d6-ca8d-4dcf-b55a-c08f760772d0',
  'scryfallOracleId': '21254bdb-7b1f-4ac6-8fdd-f0ac2c35f47d',
  'tcgplayerProductId': '276720'},
 'isReprint': True,
 'isStarter': True,
 'keywords': ['Delirium', 'Trample'],
 'language': 'English',
 'layout': 'normal',
 'legalities': {'commander': 'Legal',
  'duel': 'Legal',
  'legacy': 'Legal',
  'modern': 'Legal',
  'pi

In [6]:
df

,meta,data
date,2022-07-12,NaN
version,5.2.0+20220712,NaN
10E,NaN,"{'baseSetSize': 383, 'block': 'Core Set', 'boo..."
2ED,NaN,"{'baseSetSize': 302, 'block': 'Core Set', 'boo..."
2X2,NaN,"{'baseSetSize': 577, 'cards': [{'artist': 'Mar..."
...,...,...
YSNC,NaN,"{'baseSetSize': 30, 'block': 'Alchemy 2022', '..."
ZEN,NaN,"{'baseSetSize': 249, 'block': 'Zendikar', 'boo..."
ZNC,NaN,"{'baseSetSize': 1, 'block': 'Commander', 'card..."
ZNE,NaN,"{'baseSetSize': 30, 'cards': [{'artist': 'Adam..."
